In [1]:
import torch as th
import torch.nn as nn
import syft as sy

hook = sy.TorchHook(th)

alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
charlie = sy.VirtualWorker(hook, id="charlie")
dan = sy.VirtualWorker(hook, id="dan")
james = dan

hook.local_worker.is_client_worker = False
me = hook.local_worker

import sys
torch = th
syft = sy

In [5]:
from syft.workers.abstract import AbstractWorker

In [2]:
me._objects

{}

In [16]:
me.clear_objects()
x = torch.zeros([1, 2])
x = th.tensor([1, 2.])#.send(bob)
y = x + x

In [17]:
me._objects

{10793187173: tensor([1., 2.])}

In [3]:
# Fetch plan
fetched_plan = plan.owner.fetch_plan(plan.id, james, copy=True)

# Execute the fetch plan
x = th.tensor([-1.0])
x_sh = x.fix_precision().share(alice, bob, crypto_provider=charlie)
decrypted = fetched_plan(x_sh).get().float_prec()

# Compare with local plan
plan.get().get().float_precision()
expected = plan(x)
assert th.all(decrypted - expected.detach() < 1e-2)
assert fetched_plan.state_ids != plan.state_ids

# Make sure fetched_plan is using the readable_plan
assert fetched_plan.forward is None
assert fetched_plan.is_built

# Make sure plan is using the blueprint: forward
assert plan.forward is not None



fetch plan remote
((26, (1, ((6, ((5, (b'torch.nn.functional.linear',)), None, (6, ((20, (49453053638, 69422376521, b'me', None, (10, (1,)), True)), (20, (64512595861, 38535999723, b'me', None, (10, (1, 1)), True)), (20, (52672421927, 43403017102, b'me', None, (10, (1,)), True)))), (0, ()))), (1488124967,)))),)
Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:98497037602 -> alice:98932200342]
	-> [PointerTensor | me:47539520056 -> bob:54917562180]
	*crypto provider: charlie*
Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:35474926983 -> alice:74930229730]
	-> [PointerTensor | me:50799772901 -> bob:24448019977]
	*crypto provider: charlie*
[(26, (1, ((6, ((5, (b'torch.nn.functional.linear',)), None, (6, ((20, (49453053638, 69422376521, b'me', None, (10, (1,)), True)), (20, (64512595861, 50841672071, b'me', None, (10, (1, 1)), True)), (20, (52672421927, 30977409431, b'me', None, (10, (1,)), True)))), (0, ()))), (1488124967,))))]

KeyError: 'Object "35927502544" not found on worker!!!You just tried to interact with an object ID:35927502544 on <VirtualWorker id:me #objects:56> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [ ]:
dan._objects[59244868788].child.id

In [4]:
dan.clear_objects()
x = th.tensor([-1.0])
x_sh = x.fix_precision().share_(alice, bob, crypto_provider=charlie)
p = x_sh.send_(dan)

assert p.id_at_location in dan._objects

In [5]:
plan.bias

tensor([3.], grad_fn=<AddBackward0>)

In [4]:
for p in plan.parameters():
    print(':', p)

In [3]:
print(x)
plan.get()
print(plan.fc1.weight)
plan.get()
print(plan.fc1.weight)
plan.float_precision()
print(plan.fc1.weight)
plan(x)

tensor([-1.])
Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:1965411898 -> alice:71168104813]
	-> [PointerTensor | me:39951566140 -> bob:21784104451]
	*crypto provider: charlie*
Parameter>FixedPrecisionTensor>tensor([[700]])
Parameter containing:
tensor([[0.7000]], requires_grad=True)


tensor([-1.4960], grad_fn=<AddBackward0>)

In [3]:
dan.clear_objects()

hook.local_worker.is_client_worker = False

class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 1)
        self.add_to_state(["fc1"])

    def forward(self, x):
        return self.fc1(x)

plan = Net()
plan.build(th.tensor([1.2]))
        
plan.fix_precision().share(alice, bob, crypto_provider=charlie).send(dan)

print('dan', dan._objects)
# Fetch plan
fetched_plan = plan.owner.fetch_plan(plan.id, dan)

# Execute it locally
x = th.tensor([-1.0])
print('dan', dan._objects)

print(plan.readable_plan)

x_sh = x.fix_precision().share(alice, bob, crypto_provider=charlie)

plan.get()
#print("#", plan(x))
print('#', fetched_plan(x_sh).get().float_prec())

assert fetched_plan.forward is None
assert fetched_plan.is_built

hook.local_worker.is_client_worker = True

??? tensor([[-0.2052]])
??? tensor([-0.9597])
MODULE SHARE
param Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:99221922262 -> alice:80743885303]
	-> [PointerTensor | me:35623597249 -> bob:37213709913]
	*crypto provider: charlie*
param Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:78385820910 -> alice:64523343405]
	-> [PointerTensor | me:42310564579 -> bob:39414523222]
	*crypto provider: charlie*
??? FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:99221922262 -> alice:80743885303]
	-> [PointerTensor | me:35623597249 -> bob:37213709913]
	*crypto provider: charlie*
??? FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:78385820910 -> alice:64523343405]
	-> [PointerTensor | me:42310564579 -> bob:39414523222]
	*crypto provider: charlie*
dan {52156234865: Parameter containing:
Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | dan:99221922262 -> alice:80743885303]

In [6]:
dan.clear_objects()

hook.local_worker.is_client_worker = False

class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 1)
        self.add_to_state(["fc1"])

    def forward(self, x):
        return self.fc1(x)

plan = Net()
plan.build(th.tensor([1.2]))
        
plan.send(dan)#.fix_precision().share(alice, bob, crypto_provider=charlie)
# Fetch plan
fetched_plan = plan.owner.fetch_plan(plan.id, dan)

# Execute it locally
x = th.tensor([-1.0])
print('dan', dan._objects)

print(plan.readable_plan)

#x_sh = x.fix_precision().share(alice, bob, crypto_provider=charlie)

plan.get()
print("#", plan(x))
print('#', fetched_plan(x))

assert fetched_plan.forward is None
assert fetched_plan.is_built

hook.local_worker.is_client_worker = True

Parameter containing:
tensor([[0.6982]], requires_grad=True)
Parameter containing:
tensor([-0.6616], requires_grad=True)
dan {71190159252: <Plan Net id:71190159252 owner:dan built>}
((26, (1, ((6, ((5, (b'torch.nn.functional.linear',)), None, (6, ((20, (38067852273, 38802502033, 'me', None, (10, (1,)), True)), (20, (58021211392, 9341255612, 'me', None, (10, (1, 1)), True)), (20, (91235249738, 82417091800, 'me', None, (10, (1,)), True)))), (0, ()))), (64788480055,)))),)
# tensor([-1.3598], grad_fn=<AddBackward0>)
[(26, (1, ((6, ((5, (b'torch.nn.functional.linear',)), None, (6, ((20, (38067852273, 31309484437, b'me', None, (10, (1,)), True)), (20, (58021211392, 49680266793, b'me', None, (10, (1, 1)), True)), (20, (91235249738, 90183594872, b'me', None, (10, (1,)), True)))), (0, ()))), (58740791423,))))]
# tensor([-1.3598], requires_grad=True)


In [4]:
hook.local_worker.is_client_worker = False

@sy.func2plan(args_shape=[(1,)], state={"bias": th.tensor([1.0])})
def plan_mult_3(data, state):
    bias = state.read('bias')
    return data * 3 + bias

plan_mult_3.send(bob)

print(plan_mult_3)

# Fetch plan
fetched_plan = plan_mult_3.owner.fetch_ref_to_plan(plan_mult_3.id, bob)
print(fetched_plan.readable_plan)
print('####', fetched_plan.arg_ids)

# Execute it locally
x = th.tensor([-1., 2, 3]).send(bob)
print(x)
#assert (plan_mult_3(x) == th.tensor([-2., 7, 10])).all()
print('****', fetched_plan.locations)
assert (fetched_plan(x).get() == th.tensor([-2., 7, 10])).all()
assert fetched_plan.forward is None
assert fetched_plan.is_built

hook.local_worker.is_client_worker = True

<Plan plan_mult_3 id:82919460802 owner:me location:bob built>
state ptr [PointerTensor | me:89848703718 -> bob:39427126860]
((26, (1, ((6, ((5, (b'__mul__',)), (20, (4255184338, 80606786064, b'bob', None, (10, (1,)), True)), (6, (3,)), (0, ()))), (74316457890,)))), (26, (1, ((6, ((5, (b'__add__',)), (20, (13701487449, 74316457890, b'bob', None, None, True)), (6, ((20, (33677974586, 39427126860, b'bob', None, (10, (1,)), True)),)), (0, ()))), (23598531311,)))))
#### [80606786064]
(Wrapper)>[PointerTensor | me:7594108288 -> bob:31544976507]
**** []
[(26, (1, ((6, ((5, (b'__mul__',)), (20, (4255184338, 31544976507, b'bob', None, (10, (1,)), True)), (6, (3,)), (0, ()))), (74316457890,)))), (26, (1, ((6, ((5, (b'__add__',)), (20, (13701487449, 74316457890, b'bob', None, None, True)), (6, ((20, (33677974586, 39427126860, b'bob', None, (10, (1,)), True)),)), (0, ()))), (5628588383,))))]
CMD me __mul__
CMD bob __mul__
tensor([-3.,  6.,  9.]) (7967966023,)
[PointerTensor | me:85709100565 -> bob

KeyError: 'Object "74316457890" not found on worker!!!You just tried to interact with an object ID:74316457890 on <VirtualWorker id:bob #objects:8> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [3]:
bob._objects

{12509956793: tensor([1.]),
 6388607795: <Plan plan_mult_3 id:6388607795 owner:bob built>,
 69329147363: tensor([-3.,  6.,  9.])}

In [2]:
hook.local_worker.is_client_worker = False

@sy.func2plan(args_shape=[(1,)], state={"bias": th.tensor([1.0])})
def plan_mult_3(data, state):
    bias = state.read('bias')
    return data * 3 + bias

plan_mult_3.send(bob)

print(plan_mult_3)

# Fetch plan
fetched_plan = plan_mult_3.owner.fetch_plan(plan_mult_3.id, bob)
print(fetched_plan.readable_plan)

# Execute it locally
x = th.tensor([-1., 2, 3])
assert (plan_mult_3(x) == th.tensor([-2., 7, 10])).all()
assert (fetched_plan(x) == th.tensor([-2., 7, 10])).all()
assert fetched_plan.forward is None
assert fetched_plan.is_built


hook.local_worker.is_client_worker = True

<Plan plan_mult_3 id:77601206298 owner:me location:bob built>
[(26, (1, ((6, ((5, (b'__mul__',)), (20, (549567130, 49430313246, b'me', None, (10, (1,)), True)), (6, (3,)), (0, ()))), (31674662051,)))), (26, (1, ((6, ((5, (b'__add__',)), (20, (26491873883, 31674662051, b'me', None, None, True)), (6, ((20, (24155885429, 36448527626, b'me', None, (10, (1,)), True)),)), (0, ()))), (95386143246,))))]


In [2]:
bob.clear_objects()

bob.log_msgs = True

x = th.tensor([1, 2, 3, 4]).send(bob)

id_on_worker = x.id_at_location

assert id_on_worker in bob._objects

del x  # this is the test
assert isinstance(bob._get_msg(-1), sy.messaging.ForceObjectDeleteMessage)

In [4]:
x = th.tensor([1, 2, 3, 4, 5])
type(x.fix_prec().child)

syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor

In [12]:
t = torch.tensor([1.,2.,3.])
t.shape                         # torch.Size([3])
bds = sy.BaseDataset(t, t)
bds.send(bob)
bds.data.shape  

> grad chack
* 28287641614 (Wrapper)>[PointerTensor | me:28287641614 -> bob:18947235688]
*g 28287641614 (Wrapper)>[PointerTensor | me:28287641614 -> bob:18947235688]::grad
*stored None
< grad chack None


torch.Size([3])

In [5]:
x = th.tensor([1, 2, 3, 4, 5])
x

y = x + x

print(y)

import syft as sy

# commented out because the test needs to use the global one
# hook = sy.TorchHook(th)

th.tensor([1, 2, 3, 4, 5])

bob = sy.VirtualWorker(hook, id="bob_udacity")

bob._objects

x = th.tensor([1, 2, 3, 4, 5])

x = x.send(bob)

bob._objects

assert len(bob._objects) == 1

x.location

x.id_at_location

x.id

x.owner

hook.local_worker

x

x = x.get()
x

bob._objects

assert len(bob._objects) == 0

alice = sy.VirtualWorker(hook, id="alice_udacity")

x = th.tensor([1, 2, 3, 4, 5])

x_ptr = x.send(bob, alice)

x_ptr.get()

x = th.tensor([1, 2, 3, 4, 5]).send(bob, alice)

x.get(sum_results=True)

x = th.tensor([1, 2, 3, 4, 5]).send(bob)
y = th.tensor([1, 1, 1, 1, 1]).send(bob)

x
y

z = x + y

z

z = z.get()
z

z = th.add(x, y)
z

z = z.get()
z

x = th.tensor([1.0, 2, 3, 4, 5], requires_grad=True).send(bob)
y = th.tensor([1.0, 1, 1, 1, 1], requires_grad=True).send(bob)

z = (x + y).sum()

z.backward()

x = x.get()

x

x.grad

input = th.tensor([[1.0, 1], [0, 1], [1, 0], [0, 0]], requires_grad=True).send(bob)
target = th.tensor([[1.0], [1], [0], [0]], requires_grad=True).send(bob)

weights = th.tensor([[0.0], [0.0]], requires_grad=True).send(bob)

for i in range(10):
    pred = input.mm(weights)

    loss = ((pred - target) ** 2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

bob = bob.clear_objects()

assert len(bob._objects) == 0

x = th.tensor([1, 2, 3, 4, 5]).send(bob)

assert len(bob._objects) == 1

del x

assert len(bob._objects) == 0

x = th.tensor([1, 2, 3, 4, 5]).send(bob)

assert len(bob._objects) == 1

x = "asdf"

assert len(bob._objects) == 0

x = th.tensor([1, 2, 3, 4, 5]).send(bob)

bob._objects

x = "asdf"

bob._objects

del x

bob._objects

bob = bob.clear_objects()
bob._objects

for i in range(1000):
    x = th.tensor([1, 2, 3, 4, 5]).send(bob)

assert len(bob._objects) == 1

x = th.tensor([1, 2, 3, 4, 5]).send(bob)
y = th.tensor([1, 1, 1, 1, 1])

# throws error
# z = x + y

x = th.tensor([1, 2, 3, 4, 5]).send(bob)
y = th.tensor([1, 1, 1, 1, 1]).send(alice)

# throws error
# z = x + y

from torch import nn, optim

# A Toy Dataset
data = th.tensor([[1.0, 1], [0, 1], [1, 0], [0, 0]], requires_grad=True)
target = th.tensor([[1.0], [1], [0], [0]], requires_grad=True)

# A Toy Model
model = nn.Linear(2, 1)

opt = optim.SGD(params=model.parameters(), lr=0.1)

def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target) ** 2).sum()

        loss.backward()

        opt.step()

        print(loss.data)

train()

data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

datasets = [(data_bob, target_bob), (data_alice, target_alice)]

def train(iterations=20):

    model = nn.Linear(2, 1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)

    for iter in range(iterations):

        for _data, _target in datasets:
            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target) ** 2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

train()

bob.clear_objects()
alice.clear_objects()

x = th.tensor([1, 2, 3, 4, 5]).send(bob)

x = x.send(alice)

bob._objects

alice._objects

y = x + x

y

bob._objects

alice._objects

jon = sy.VirtualWorker(hook, id="jon")

bob.clear_objects()
alice.clear_objects()

x = th.tensor([1, 2, 3, 4, 5]).send(bob).send(alice)

bob._objects

alice._objects

x = x.get()
x

bob._objects

alice._objects

x = x.get()
x

bob._objects

bob.clear_objects()
alice.clear_objects()

x = th.tensor([1, 2, 3, 4, 5]).send(bob).send(alice)

bob._objects

alice._objects

del x

bob._objects

alice._objects

bob.clear_objects()
alice.clear_objects()

x = th.tensor([1, 2, 3, 4, 5]).send(bob)

bob._objects

alice._objects

x.move(alice)

bob._objects

alice._objects

x = th.tensor([1, 2, 3, 4, 5]).send(bob).send(alice)

bob._objects

alice._objects

x.remote_get()

bob._objects

alice._objects

x.move(bob)

x

bob._objects

alice._objects

tensor([ 2,  4,  6,  8, 10])
tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


AssertionError: 

In [3]:
me.clear_objects()
alice.clear_objects()
bob.clear_objects()

t = th.tensor([1, 2])
crt = t.fix_precision()

to_alice = crt.send(alice)
to_alice_id = to_alice.id_at_location

print('\n---')
#print(me._objects)
print(alice._objects)
print(bob._objects)

assert to_alice_id in alice._objects

to_bob_to_alice = to_alice.send(bob)
to_bob_to_alice_id = to_bob_to_alice.id_at_location


print('\n---')
#print(me._objects)
print(alice._objects)
print(bob._objects)

assert to_alice_id in alice._objects
assert to_bob_to_alice_id in bob._objects

to_alice_back = to_bob_to_alice.get()


print('\n---')
#print(me._objects)
print(alice._objects)
print(bob._objects)

print(to_alice_back)

assert to_bob_to_alice_id not in bob._objects

alice
+ {}

---
{32267512132: (Wrapper)>FixedPrecisionTensor>tensor([1000, 2000])}
{}
> grad chack
alice
+ {}
* 23766904668 (Wrapper)>[PointerTensor | me:23766904668 -> alice:32267512132]
*g 23766904668 [PointerTensor | me:23766904668 -> alice:32267512132]::grad
*stored (Wrapper)>[PointerTensor | me:23766904668 -> alice:32267512132]
< grad chack None
bob
+ {23766904668: (Wrapper)>[PointerTensor | bob:23766904668 -> alice:32267512132]}

---
{32267512132: (Wrapper)>FixedPrecisionTensor>tensor([1000, 2000])}
{23766904668: (Wrapper)>[PointerTensor | bob:23766904668 -> alice:32267512132]}
bob
> grad chack
alice
+ {}
* 23766904668 (Wrapper)>[PointerTensor | bob:23766904668 -> alice:32267512132]
*g 23766904668 [PointerTensor | bob:23766904668 -> alice:32267512132]::grad
*stored (Wrapper)>[PointerTensor | me:23766904668 -> alice:32267512132]
< grad chack None
+ {}

---
{32267512132: (Wrapper)>FixedPrecisionTensor>tensor([1000, 2000])}
{}
(Wrapper)>[PointerTensor | me:23766904668 -> alice:32267

In [ ]:
me.clear_objects()
bob.clear_objects()
alice.clear_objects()

t = th.tensor([5.])
p = t.send(bob)
print('---')
#print(me._objects)
print(bob._objects)
print(alice._objects)

p2 = p.send(alice)

print('---')
#print(me._objects)
print(bob._objects)
print(alice._objects)

p2.get().get()

In [3]:
me.clear_objects()
for i in range(2):
    #x = th.randint(1, 4, (1,))
    x = th.Tensor([1])
    #p = x.send(bob)
    y = x + 1
    print(me._objects)

{}
{}


LOCAL AST

In [6]:
me.clear_objects()
bob.clear_objects()
t = th.tensor([5.])
print('---')
print(me._objects)
print(bob._objects)
fp = t.fix_precision()
print('&&&', fp.owner.id, fp.child.owner.id)
ast = fp.share(bob, charlie, crypto_provider=dan)
print('---')
print(me._objects)
print(bob._objects)

---
{70332406668: tensor([5.])}
{}
&&& me me
---
{70332406668: (Wrapper)>FixedPrecisionTensor>tensor([5000]), 41110726330: (Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:85784121451 -> bob:44940711548]
	-> [PointerTensor | me:25740312493 -> charlie:2790009857]
	*crypto provider: dan*, 17985967760: tensor([3135903654160558164]), 66956636999: tensor([1475782364266834740])}
{44940711548: tensor([3135903654160558164])}


In [7]:
me.clear_objects()
bob.clear_objects()
t = th.tensor([5.])
p = t.send(bob)
print('---')
#print(me._objects)
print(bob._objects)
fp = p.fix_precision()
ast = fp.share(alice, charlie, crypto_provider=dan)
print('---')
#print(me._objects)
print(bob._objects)

print(ast.get().get().float_prec())

---
{54830710849: tensor([5.])}
---
{66736858899: (Wrapper)>FixedPrecisionTensor>tensor([5000]), 12308757689: (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:63667526382 -> alice:70547979334]
	-> [PointerTensor | me:50602148300 -> charlie:67161374735]
	*crypto provider: dan*}
tensor([5.])


In [3]:
t = th.tensor([1.])
x = t.send(dan).fix_precision().share(bob, charlie, crypto_provider=dan)

wrap wrap
wrap wrap
% {91263851843: (Wrapper)>FixedPrecisionTensor>tensor([5000]), 2888401262: tensor([1.]), 77812153775: tensor([1.]), 92405367534: (Wrapper)>[PointerTensor | me:92405367534 -> dan:2888401262]}
85107129125
wrap wrap


In [4]:
dan._objects

{77274839854: (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:13334052158 -> bob:11183558852]
 	-> [PointerTensor | me:39205664941 -> charlie:32614949670]
 	*crypto provider: dan*}

In [7]:
@sy.func2plan(args_shape=[(1,)])
def plan_mult_3(data):
    return data * 3

plan_mult_3.send(alice)

# Fetch plan
fetched_plan = alice.fetch_plan(plan_mult_3.id)

hook.local_worker.clear_objects()

t = th.tensor([1.])
#p = t.send(bob)
#fetched_plan.fix_precision().share(bob, charlie, crypto_provider=dan)
x = t.fix_precision().share(bob, charlie, crypto_provider=dan)

print('original', t.id)
print('AST', x.id)
print('Shares', [(p.id, p.id_at_location)  for k, p in x.child.child.child.items()])

print(hook.local_worker._objects)
print(fetched_plan.readable_plan)
# Execute it with an AST
print(fetched_plan(x))

original 52903712819
AST 12201223335
Shares [(11566043928, 1658108377), (52116868672, 91318267980)]
{52903712819: tensor([1.]), 19782221745: tensor(4611686018427387904), 19952750278: tensor([3474844053801700524]), 98020423652: tensor([1136841964625688380])}
((26, (1, ((6, ((5, (b'__mul__',)), (20, (84319371425, 99622639975, b'me', None, (10, (1,)), True)), (6, (3,)), (0, ()))), (55229073736,)))),)
[(26, (1, ((6, ((5, (b'__mul__',)), (20, (84319371425, 12201223335, b'me', None, (10, (1,)), True)), (6, (3,)), (0, ()))), (88017635045,))))]


KeyError: 'Object "12201223335" not found on worker!!!You just tried to interact with an object ID:12201223335 on <VirtualWorker id:me #objects:4> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

Pointers

In [7]:
@sy.func2plan(args_shape=[(1,)])
def plan_mult_3(data):
    return data * 3

sent_plan = plan_mult_3.send(alice)

# Fetch plan
fetched_plan = alice.fetch_plan(sent_plan.id)
get_plan = sent_plan.get()

t = th.tensor([1.])
p = t.send(bob)
fetched_plan.send(bob)
#x = t.fix_precision().share(bob, charlie, crypto_provider=dan)
# Execute it with an AST
print(fetched_plan(p).get())

tensor([3.])
